# Лабораторная работа №1

## Библиотеки
В ходе всех лабораторных работ будут использоваться следующие библиотеки:

Библиотека|Описание
-|-
[numpy](http://www.numpy.org/)|Работа с массивами и матрицами
[pandas](http://pandas.pydata.org/)|Удобная обработка наборов данных
[scikit-learn](https://scikit-learn.org/stable/)|Алгоритмы и методы машинного обучения
 
Если для выполнения какой-либо из лабораторных работ потребуются дополнительные библиотеки, об этом будет сказано в начале тетрадки соответствующей лабораторной работы.

## Задание
Лабораторная работа состоит из нескольких этапов:
 1. Загрузить датасет в формате CSV согласно варианту.
 2. Произвести согласно варианту:
     * Унификацию данных
     * Приведение отношений к первой нормальной форме
     * Создание признаков
 3. Создать простейший линейный классификатор или регрессор и обучить его при следующих условииях:
     * По количеству признаков:
         * Для обучения использовать только те признаки, которые на Ваш взгляд больше всего влияют на выходной параметр
         * Для обучения использовать все признаки
     * По применению нормализации:
         * С использованием нормализации масштаба
         * Без использования нормализации масштаба
     * По характеру обучающих и контрольных наборов данных:
         * Обучающий набор -- все данные, контрольный -- 1/N часть общего набора
         * Обучать на (N-1)/N набора, проверять на оставшейся 1/N части. Данные не перемешивать перед разбиением!
         * То же самое, что пункт выше, но с перемешиванием данных.
         * Кросс-проверка, с разбиением данных на N наборов.
         * N -- согласно варианту
 4. Для каждого случая рассмотреть и проанализировать точность обученной модели.
 
Ниже представлена минимальная теоретическая информация, необходимая для выполнения задания, куски кода для примера, а также более детальное пояснение каждого из этапов.

## Инструкция
### Этап 1: Импорт данных
Для импорта данных удобно использовать библиотеку pandas.  
Чтобы иметь возможность использовать pandas, необходимо выполнить:  

In [1]:
import pandas as pd
from sklearn import linear_model
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np

Теперь можно импортировать данные. Так как все наборы данных для данной лабы представлены в формате CSV, можем смело пользоваться методом pandas для считывания данных в формате CSV:  

In [2]:
df = pd.read_csv('datasets/titanic.csv')

Метод `read_csv()` принимает в качестве аргумента имя файла и кучу других параметров. Подробнее о методе `read_csv()` можно прочитать [здесь](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html).  
Но что же возвращает этот метод? Он возвращает содержимое файла, упакованное в структуру данных `DataFrame`, которая является  одной из основных в библиотеке pandas.

DataFrame -- двумерная структура данных, с размеченными столбцами и строками. По сути, просто таблица с удобным интерфейсом доступа к данным. Ниже перечислены несколько удобных способов доступа к данным:  

Способ доступа|Описание|Пример
-|-|-
`df.head([n])`|Получить первые n строк|`df.head(5)`
`df[str]`|Выбрать столбец|`df['sex']`
`df.iloc[idx]`|Выбрать строку(и)|`df.iloc[2:6]`
`df.at[idx, str]`|Выбрать ячейку|`df.at[4, 'sex']`

Изменять данные можно обычным оператором присваивания (=).  
Некоторые ресурсы для дальнейшего ознакомления:
 * [Легкий для чтения гайд по структурам данных в pandas](https://pandas.pydata.org/pandas-docs/stable/getting_started/dsintro.html#dataframe)
 * [Чуть подробнее про разметку и доступ к данным](https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#indexing)
 * [Документация по DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/frame.html)

### Этап 2: Предобработка данных
Теперь, когда данные загружены в память, можно приступать к их предобработке. 
Сначала, посмотрим, что же вообще за данные там лежат:

In [3]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Видим, что данные в столбце "пол" не унифицированы. То бишь, один и тот же пол может обозоначаться как "male", "Male", "MALE" и т. д.  
Такая ситуация крайне нежелательна, и от этого нужно избавляться.  
Кроме того, модель на вход принимает только числовые, так что каждому полу нужно сопоставить некоторое целое число, и преобразовать столбец пола нужным образом.  
Более подробно о том, что именно необходимо сделать на данном этапе, см. ниже в таблице вариантов.

In [ ]:
def unify_sex(df: pd.DataFrame) -> pd.DataFrame:
    df['Sex'] = df['Sex'].apply(lambda x: 0 if x == 'male' else 1 if x == 'female' else 2)
    # Implement this
    return df

df = unify(df)
df.head()

Сделаем разбивку на тестовые и тренировочные данные

In [ ]:
# Перемешаем данные
data = np.asarray(df.sample(frac=1), dtype=np.float64)

# Определяем точку надреза
size = len(df)
test_slice = int(size / 10)

X_train = data[test_slice:, :-1]
y_train = data[test_slice:, -1]

X_test = data[:test_slice, :-1]
y_test = data[:test_slice, -1]

# Не забывайте, что в реальном задании небходимо использовать кросс-валидацию!

Проведем нормализацию масштаба данных

In [4]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",1,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",0,35.0,0,0,373450,8.0500,NaN,S


### Этап 3: Создание и обучение линейной модели

In [ ]:
# Загрузим данные
# df = pd.read_csv('datasets/boston_housing.csv')

In [ ]:
# Для задач регрессии
regressor = linear_model.LinearRegression()
regressor.fit(X=X_train_scaled, y=y_train)
# Для задач классификации
# classifier = linear_model.Perceptron()
# classifier.fit(X=X_train_scaled, y=y_train)

### Этап 4: Оценка качества модели

In [ ]:
regressor.score(X=X_test_scaled, y=y_test)
# ИЛИ
# predictions = clf.predict(X_test_scaled)
# accuracy_score(y_test, predictions)

## Варианты

| Вариант| Датасет   | Создаваемые признаки/Унификация | N   | Результат |
|-|-|-|-|-|
| 1 | mushroom.csv | -/odor | 5 | Съедобность гриба |
| 2 | boston_housing.csv | -/- | 6 | Стоимость дома |
| 3 | students.csv | Удалить первый столбец (порядковый номер)/sex | 4 | Склонность к употреблению спиртного  |
| 4 | adult.csv | -/sex | 5 | Размер зарплаты |
| 5 | wines.csv | Классифицировать содержание сахара(сухое, полусухое, ...)/- | 5 | Вкусовые качества вина |
| 6 | titanic.csv | -/sex | 3 | Выжил ли пассажир |


## Содержание отчета

1. Титульный лист
2. Текст задания согласно варианту
3. Исходный код
4. Результаты различных моделей и анализ результатов
5. Вывод по работе

## Вопросы к защите

1. Что понимается под предобработкой данных и зачем она нужна?
2. Loss function, Q function
3. Кросс-валидация
4. Метрики(precision, recall, F-measure)
5. Что понимается под признаком в терминах машинного обучения и какие виды признаков бывают?